# VRPTW Solver: Local Search + VND + Simulated Annealing + Tabu Search

This notebook implements a hybrid metaheuristic algorithm for solving Vehicle Routing Problems **with Time Windows** (VRPTW).

## Algorithm Components:
1. **Initial Solution**: Time-feasible nearest neighbor heuristic
2. **Local Search**: Time-window aware improvement operators
3. **VND (Variable Neighborhood Descent)**: Systematic neighborhood exploration
4. **Simulated Annealing**: Accept worse solutions with decreasing probability
5. **Tabu Search**: Memory structure to avoid cycling

## Key Differences from CVRP:
- **Time Windows**: Each customer has [ready_time, due_date]
- **Service Time**: Time spent at each customer location
- **Arrival Time**: Must arrive within time window
- **Wait Time**: Can wait if arriving before ready_time

## Goal:
Find feasible solutions within **7% gap** from the optimal solution.

In [1]:
# Import required libraries
import numpy as np
import vrplib
import yaml
import random
import math
import time
import os
from collections import deque
from copy import deepcopy
from typing import List, Dict, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")

Libraries imported successfully!


In [2]:
# Load configuration from YAML file
with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)

print("Configuration loaded:")
print(yaml.dump(config, default_flow_style=False))

Configuration loaded:
general:
  random_seed: 42
  time_limit_seconds: 300
  verbose: true
initial_solution:
  method: nearest_neighbor
  randomness: 0.1
local_search:
  max_iterations: 1000
  max_iterations_without_improvement: 200
quality:
  penalty_capacity_violation: 10000
  target_gap_percentage: 7.0
simulated_annealing:
  alpha: 0.95
  final_temperature: 0.1
  initial_temperature: 1000.0
  iterations_per_temperature: 100
tabu_search:
  aspiration_enabled: true
  tabu_tenure: 20
  tabu_tenure_random_range: 10
vnd:
  max_iterations_without_improvement: 50
  neighborhoods:
  - swap
  - relocate
  - two_opt
  - cross_exchange



In [3]:
# Set random seed for reproducibility
random.seed(config['general']['random_seed'])
np.random.seed(config['general']['random_seed'])

print(f"Random seed set to: {config['general']['random_seed']}")

Random seed set to: 42


## Data Structures and Helper Functions

In [4]:
class VRPTWSolution:
    """Represents a VRPTW solution with routes, cost, and time information."""
    
    def __init__(self, routes: List[List[int]], distance_matrix: np.ndarray,
                 time_windows: np.ndarray, service_times: np.ndarray):
        self.routes = routes
        self.distance_matrix = distance_matrix
        self.time_windows = time_windows  # [n_customers, 2] - [ready_time, due_date]
        self.service_times = service_times  # [n_customers] - service duration
        self.route_times = []  # Arrival times for each customer in each route
        self.cost = self.calculate_cost()
    
    def calculate_route_times(self, route: List[int]) -> List[float]:
        """Calculate arrival times for a route."""
        if not route:
            return []
        
        times = []
        current_time = 0.0
        current_location = 0  # Depot
        
        for customer in route:
            # Travel time to customer
            travel_time = self.distance_matrix[current_location, customer]
            arrival_time = current_time + travel_time
            
            # Wait if arriving before ready time
            ready_time = self.time_windows[customer, 0]
            start_service = max(arrival_time, ready_time)
            
            times.append(arrival_time)
            
            # Update current time and location
            current_time = start_service + self.service_times[customer]
            current_location = customer
        
        return times
    
    def calculate_cost(self) -> float:
        """Calculate total cost (distance) of the solution."""
        total_cost = 0
        self.route_times = []
        
        for route in self.routes:
            if len(route) == 0:
                self.route_times.append([])
                continue
            
            # Calculate times
            times = self.calculate_route_times(route)
            self.route_times.append(times)
            
            # Calculate distance
            total_cost += self.distance_matrix[0, route[0]]
            for i in range(len(route) - 1):
                total_cost += self.distance_matrix[route[i], route[i+1]]
            total_cost += self.distance_matrix[route[-1], 0]
        
        return total_cost
    
    def update_cost(self):
        """Recalculate cost after modification."""
        self.cost = self.calculate_cost()
    
    def is_time_feasible(self, route: List[int]) -> bool:
        """Check if a route satisfies time window constraints."""
        if not route:
            return True
        
        current_time = 0.0
        current_location = 0
        
        for customer in route:
            # Travel to customer
            travel_time = self.distance_matrix[current_location, customer]
            arrival_time = current_time + travel_time
            
            # Check if we can arrive before due date
            due_date = self.time_windows[customer, 1]
            if arrival_time > due_date:
                return False
            
            # Update time
            ready_time = self.time_windows[customer, 0]
            start_service = max(arrival_time, ready_time)
            current_time = start_service + self.service_times[customer]
            current_location = customer
        
        # Check if we can return to depot
        return_time = current_time + self.distance_matrix[current_location, 0]
        depot_due = self.time_windows[0, 1]
        
        return return_time <= depot_due
    
    def is_feasible(self, demands: np.ndarray, capacity: int) -> bool:
        """Check if solution satisfies both capacity and time window constraints."""
        for route in self.routes:
            # Check capacity
            route_demand = sum(demands[customer] for customer in route)
            if route_demand > capacity:
                return False
            
            # Check time windows
            if not self.is_time_feasible(route):
                return False
        
        return True
    
    def copy(self):
        """Create a deep copy of the solution."""
        return VRPTWSolution(
            [route.copy() for route in self.routes],
            self.distance_matrix,
            self.time_windows,
            self.service_times
        )

print("VRPTWSolution class defined!")

VRPTWSolution class defined!


In [5]:
def calculate_distance_matrix(instance: Dict) -> np.ndarray:
    """Calculate distance matrix from instance data."""
    if 'edge_weight' in instance:
        return instance['edge_weight']
    elif 'node_coord' in instance:
        coords = np.array(instance['node_coord'])
        n = len(coords)
        dist_matrix = np.zeros((n, n))
        for i in range(n):
            for j in range(n):
                if i != j:
                    dist = np.sqrt(np.sum((coords[i] - coords[j])**2))
                    dist_matrix[i, j] = dist
        return dist_matrix
    else:
        raise ValueError("Instance must have either 'edge_weight' or 'node_coord'")

print("Distance matrix calculation function defined!")

Distance matrix calculation function defined!


## Initial Solution Construction

In [21]:
def nearest_neighbor_solution_tw(instance: Dict, distance_matrix: np.ndarray,
                                  time_windows: np.ndarray, service_times: np.ndarray) -> VRPTWSolution:
    """Create initial solution using time-window aware nearest neighbor heuristic."""
    # Get dimension from array length (Solomon format doesn't have 'dimension' key)
    demands = np.array(instance['demand'])
    n_customers = len(demands) - 1  # Exclude depot
    capacity = instance['capacity']
    
    unvisited = set(range(1, n_customers + 1))
    routes = []
    
    while unvisited:
        route = []
        current_load = 0
        current_time = 0.0
        current_location = 0
        
        while True:
            best_customer = None
            best_score = float('inf')
            
            for customer in unvisited:
                # Check capacity
                if current_load + demands[customer] > capacity:
                    continue
                
                # Check time feasibility
                travel_time = distance_matrix[current_location, customer]
                arrival_time = current_time + travel_time
                due_date = time_windows[customer, 1]
                
                if arrival_time > due_date:
                    continue
                
                # Check if we can return to depot
                ready_time = time_windows[customer, 0]
                start_service = max(arrival_time, ready_time)
                finish_time = start_service + service_times[customer]
                return_time = finish_time + distance_matrix[customer, 0]
                depot_due = time_windows[0, 1]
                
                if return_time > depot_due:
                    continue
                
                # Calculate score (distance + time urgency)
                dist = distance_matrix[current_location, customer]
                urgency = due_date - arrival_time  # How much time left
                score = dist - 0.1 * urgency  # Prioritize urgent customers
                
                # Add randomness
                randomness = config['initial_solution']['randomness']
                score *= (1 + randomness * random.random())
                
                if score < best_score:
                    best_score = score
                    best_customer = customer
            
            if best_customer is None:
                break
            
            # Add customer to route
            route.append(best_customer)
            current_load += demands[best_customer]
            
            # Update time
            travel_time = distance_matrix[current_location, best_customer]
            arrival_time = current_time + travel_time
            ready_time = time_windows[best_customer, 0]
            start_service = max(arrival_time, ready_time)
            current_time = start_service + service_times[best_customer]
            current_location = best_customer
            
            unvisited.remove(best_customer)
        
        if route:
            routes.append(route)
    
    return VRPTWSolution(routes, distance_matrix, time_windows, service_times)

print("Time-window aware nearest neighbor function defined!")

Time-window aware nearest neighbor function defined!


## Neighborhood Operators (Time-Window Aware)

In [7]:
def swap_operator_tw(solution: VRPTWSolution, demands: np.ndarray, capacity: int) -> Optional[VRPTWSolution]:
    """Swap two customers between different routes (time-window aware)."""
    if len(solution.routes) < 2:
        return None
    
    best_solution = None
    best_cost = solution.cost
    
    for i in range(len(solution.routes)):
        for j in range(i + 1, len(solution.routes)):
            route_i = solution.routes[i]
            route_j = solution.routes[j]
            
            if len(route_i) == 0 or len(route_j) == 0:
                continue
            
            for pos_i in range(len(route_i)):
                for pos_j in range(len(route_j)):
                    new_solution = solution.copy()
                    
                    # Swap customers
                    new_solution.routes[i][pos_i], new_solution.routes[j][pos_j] = \
                        new_solution.routes[j][pos_j], new_solution.routes[i][pos_i]
                    
                    # Check feasibility (capacity + time windows)
                    if new_solution.is_feasible(demands, capacity):
                        new_solution.update_cost()
                        if new_solution.cost < best_cost:
                            best_cost = new_solution.cost
                            best_solution = new_solution
    
    return best_solution

print("Swap operator (TW) defined!")

Swap operator (TW) defined!


In [8]:
def relocate_operator_tw(solution: VRPTWSolution, demands: np.ndarray, capacity: int) -> Optional[VRPTWSolution]:
    """Relocate a customer from one route to another (time-window aware)."""
    best_solution = None
    best_cost = solution.cost
    
    for i in range(len(solution.routes)):
        route_i = solution.routes[i]
        if len(route_i) == 0:
            continue
            
        for pos_i in range(len(route_i)):
            for j in range(len(solution.routes)):
                if i == j:
                    continue
                
                for pos_j in range(len(solution.routes[j]) + 1):
                    new_solution = solution.copy()
                    
                    # Remove and insert
                    removed = new_solution.routes[i].pop(pos_i)
                    new_solution.routes[j].insert(pos_j, removed)
                    
                    # Check feasibility
                    if new_solution.is_feasible(demands, capacity):
                        new_solution.update_cost()
                        if new_solution.cost < best_cost:
                            best_cost = new_solution.cost
                            best_solution = new_solution
    
    return best_solution

print("Relocate operator (TW) defined!")

Relocate operator (TW) defined!


In [9]:
def two_opt_operator_tw(solution: VRPTWSolution, demands: np.ndarray, capacity: int) -> Optional[VRPTWSolution]:
    """Apply 2-opt improvement within each route (time-window aware)."""
    best_solution = None
    best_cost = solution.cost
    
    for route_idx in range(len(solution.routes)):
        route = solution.routes[route_idx]
        if len(route) < 2:
            continue
        
        for i in range(len(route) - 1):
            for j in range(i + 1, len(route)):
                new_solution = solution.copy()
                
                # Reverse segment
                new_solution.routes[route_idx][i:j+1] = list(reversed(new_solution.routes[route_idx][i:j+1]))
                
                # Check time feasibility
                if new_solution.is_time_feasible(new_solution.routes[route_idx]):
                    new_solution.update_cost()
                    if new_solution.cost < best_cost:
                        best_cost = new_solution.cost
                        best_solution = new_solution
    
    return best_solution

print("2-opt operator (TW) defined!")

2-opt operator (TW) defined!


## Variable Neighborhood Descent (VND)

In [10]:
def vnd_tw(solution: VRPTWSolution, demands: np.ndarray, capacity: int) -> VRPTWSolution:
    """Variable Neighborhood Descent for VRPTW."""
    neighborhoods = {
        'swap': swap_operator_tw,
        'relocate': relocate_operator_tw,
        'two_opt': two_opt_operator_tw
    }
    
    neighborhood_order = ['swap', 'relocate', 'two_opt']
    max_no_improve = config['vnd']['max_iterations_without_improvement']
    
    current_solution = solution
    k = 0
    no_improve_count = 0
    
    while k < len(neighborhood_order) and no_improve_count < max_no_improve:
        neighborhood_name = neighborhood_order[k]
        operator = neighborhoods[neighborhood_name]
        
        new_solution = operator(current_solution, demands, capacity)
        
        if new_solution is not None and new_solution.cost < current_solution.cost:
            current_solution = new_solution
            k = 0
            no_improve_count = 0
        else:
            k += 1
            no_improve_count += 1
    
    return current_solution

print("VND (TW) function defined!")

VND (TW) function defined!


## Tabu Search

In [11]:
class TabuList:
    """Tabu list for tracking forbidden moves."""
    
    def __init__(self, tenure: int):
        self.tenure = tenure
        self.tabu_dict = {}
        self.current_iteration = 0
    
    def add(self, move: Tuple, tenure_variation: int = 0):
        actual_tenure = self.tenure + random.randint(-tenure_variation, tenure_variation)
        self.tabu_dict[move] = self.current_iteration + actual_tenure
    
    def is_tabu(self, move: Tuple) -> bool:
        if move not in self.tabu_dict:
            return False
        return self.tabu_dict[move] > self.current_iteration
    
    def increment_iteration(self):
        self.current_iteration += 1
        expired = [move for move, expiration in self.tabu_dict.items() 
                   if expiration <= self.current_iteration]
        for move in expired:
            del self.tabu_dict[move]

print("TabuList class defined!")

TabuList class defined!


## Simulated Annealing with Tabu Search

In [12]:
def acceptance_probability(current_cost: float, new_cost: float, temperature: float) -> float:
    """Calculate acceptance probability for SA."""
    if new_cost < current_cost:
        return 1.0
    return math.exp((current_cost - new_cost) / temperature)

def simulated_annealing_with_tabu_tw(
    initial_solution: VRPTWSolution, 
    demands: np.ndarray, 
    capacity: int,
    time_limit: float = None
) -> Tuple[VRPTWSolution, List[float]]:
    """Hybrid Simulated Annealing with Tabu Search for VRPTW."""
    
    # Parameters from config
    temp = config['simulated_annealing']['initial_temperature']
    final_temp = config['simulated_annealing']['final_temperature']
    alpha = config['simulated_annealing']['alpha']
    iterations_per_temp = config['simulated_annealing']['iterations_per_temperature']
    
    tabu_tenure = config['tabu_search']['tabu_tenure']
    tabu_tenure_variation = config['tabu_search']['tabu_tenure_random_range']
    aspiration_enabled = config['tabu_search']['aspiration_enabled']
    
    max_iterations = config['local_search']['max_iterations']
    max_no_improve = config['local_search']['max_iterations_without_improvement']
    
    # Initialize
    current_solution = initial_solution
    best_solution = current_solution.copy()
    tabu_list = TabuList(tabu_tenure)
    
    cost_history = [best_solution.cost]
    no_improve_count = 0
    total_iterations = 0
    
    start_time = time.time()
    
    # Neighborhood operators
    operators = [swap_operator_tw, relocate_operator_tw, two_opt_operator_tw]
    
    while temp > final_temp and total_iterations < max_iterations:
        if time_limit and (time.time() - start_time) > time_limit:
            break
        
        if no_improve_count >= max_no_improve:
            break
        
        for _ in range(iterations_per_temp):
            # Apply VND periodically
            if total_iterations % 50 == 0:
                current_solution = vnd_tw(current_solution, demands, capacity)
            
            # Select random operator
            operator = random.choice(operators)
            new_solution = operator(current_solution, demands, capacity)
            
            if new_solution is None:
                continue
            
            # Create move identifier
            move_id = (operator.__name__, hash(str(new_solution.routes)))
            
            # Check tabu status
            is_tabu = tabu_list.is_tabu(move_id)
            
            # Aspiration criterion
            aspiration = aspiration_enabled and new_solution.cost < best_solution.cost
            
            # Decide acceptance
            if (not is_tabu or aspiration):
                if random.random() < acceptance_probability(current_solution.cost, new_solution.cost, temp):
                    current_solution = new_solution
                    tabu_list.add(move_id, tabu_tenure_variation)
                    
                    # Update best solution
                    if current_solution.cost < best_solution.cost:
                        best_solution = current_solution.copy()
                        cost_history.append(best_solution.cost)
                        no_improve_count = 0
                        if config['general']['verbose']:
                            print(f"  New best: {best_solution.cost:.2f} at iteration {total_iterations}")
                    else:
                        no_improve_count += 1
            
            tabu_list.increment_iteration()
            total_iterations += 1
        
        # Cool down
        temp *= alpha
    
    return best_solution, cost_history

print("Simulated Annealing with Tabu Search (TW) function defined!")

Simulated Annealing with Tabu Search (TW) function defined!


## Main Solver Function

In [20]:
def solve_vrptw(instance_path: str) -> Dict:
    """Solve a VRPTW instance and return results."""
    
    print(f"\n{'='*60}")
    print(f"Solving: {os.path.basename(instance_path)}")
    print(f"{'='*60}")
    
    # Read instance (Solomon format)
    instance = vrplib.read_instance(instance_path, instance_format='solomon')
    
    # Extract data
    distance_matrix = calculate_distance_matrix(instance)
    demands = np.array(instance['demand'])
    capacity = instance['capacity']
    
    # Time windows: [ready_time, due_date]
    time_windows = np.array(instance['time_window'])
    service_times = np.array(instance['service_time'])
    
    # Get dimension from array length (Solomon format doesn't have 'dimension' key)
    n_nodes = len(demands)
    
    print(f"Dimension: {n_nodes} nodes")
    print(f"Capacity: {capacity}")
    print(f"Time horizon: [0, {time_windows[0, 1]:.0f}]")
    
    # Create initial solution
    print("\nCreating initial solution...")
    initial_solution = nearest_neighbor_solution_tw(instance, distance_matrix, time_windows, service_times)
    print(f"Initial cost: {initial_solution.cost:.2f}")
    print(f"Initial routes: {len(initial_solution.routes)}")
    print(f"Time feasible: {initial_solution.is_feasible(demands, capacity)}")
    
    # Apply VND
    print("\nApplying VND...")
    improved_solution = vnd_tw(initial_solution, demands, capacity)
    print(f"After VND cost: {improved_solution.cost:.2f}")
    
    # Apply SA with Tabu
    print("\nApplying Simulated Annealing with Tabu Search...")
    time_limit = config['general']['time_limit_seconds']
    start_time = time.time()
    
    final_solution, cost_history = simulated_annealing_with_tabu_tw(
        improved_solution, demands, capacity, time_limit
    )
    
    elapsed_time = time.time() - start_time
    
    print(f"\nFinal cost: {final_solution.cost:.2f}")
    print(f"Final routes: {len(final_solution.routes)}")
    print(f"Time elapsed: {elapsed_time:.2f} seconds")
    print(f"Time feasible: {final_solution.is_feasible(demands, capacity)}")
    
    # Read optimal solution if available
    optimal_cost = None
    gap_percentage = None
    
    sol_path = instance_path.replace('.txt', '.sol')
    if os.path.exists(sol_path):
        try:
            optimal_solution = vrplib.read_solution(sol_path)
            optimal_cost = optimal_solution['cost']
            gap_percentage = ((final_solution.cost - optimal_cost) / optimal_cost) * 100
            
            print(f"\nOptimal cost: {optimal_cost:.2f}")
            print(f"Gap: {gap_percentage:.2f}%")
            
            if gap_percentage <= config['quality']['target_gap_percentage']:
                print(f"✓ Solution within target gap of {config['quality']['target_gap_percentage']}%")
            else:
                print(f"✗ Solution exceeds target gap of {config['quality']['target_gap_percentage']}%")
        except Exception as e:
            print(f"Could not read optimal solution: {e}")
    
    return {
        'instance_name': os.path.basename(instance_path),
        'solution': final_solution,
        'cost': final_solution.cost,
        'optimal_cost': optimal_cost,
        'gap_percentage': gap_percentage,
        'n_routes': len(final_solution.routes),
        'time_seconds': elapsed_time,
        'cost_history': cost_history
    }

print("Main VRPTW solver function defined!")

Main VRPTW solver function defined!


## Save Solution to File

In [14]:
def save_solution_tw(result: Dict, output_dir: str = 'solutions_TW'):
    """Save VRPTW solution to a file."""
    os.makedirs(output_dir, exist_ok=True)
    
    instance_name = result['instance_name'].replace('.txt', '')
    output_path = os.path.join(output_dir, f"{instance_name}_computed.sol")
    
    routes = result['solution'].routes
    
    # Write solution
    with open(output_path, 'w') as f:
        for idx, route in enumerate(routes, 1):
            route_str = ' '.join(map(str, route))
            f.write(f"Route #{idx}: {route_str}\n")
        f.write(f"Cost {result['cost']:.1f}\n")
    
    print(f"Solution saved to: {output_path}")
    return output_path

print("Save solution (TW) function defined!")

Save solution (TW) function defined!


## Test on Solomon Instances

In [15]:
# Get list of Solomon VRPTW instances
solomon_dir = 'data/cvrplib/Vrp-Set-Solomon'
if os.path.exists(solomon_dir):
    vrptw_instances = [
        os.path.join(solomon_dir, f) for f in os.listdir(solomon_dir) 
        if f.endswith('.txt')
    ]
else:
    # Try alternative location
    vrptw_instances = [
        os.path.join('data', f) for f in os.listdir('data') 
        if f.endswith('.txt') and not f.startswith('empty')
    ]

# Select a subset for testing (different types: C, R, RC)
selected_instances = [
    inst for inst in vrptw_instances 
    if any(name in inst for name in ['C101', 'R101', 'RC101'])
][:3]  # Limit to 3 instances

print(f"Found {len(vrptw_instances)} VRPTW instances")
print(f"Selected {len(selected_instances)} for testing:")
for inst in selected_instances:
    print(f"  - {os.path.basename(inst)}")

Found 56 VRPTW instances
Selected 3 for testing:
  - C101.txt
  - R101.txt
  - RC101.txt


In [22]:
# Solve selected VRPTW instances
results = []

for instance_path in selected_instances:
    try:
        result = solve_vrptw(instance_path)
        save_solution_tw(result)
        results.append(result)
    except Exception as e:
        print(f"Error solving {instance_path}: {e}")
        import traceback
        traceback.print_exc()
        continue

print(f"\n\nCompleted solving {len(results)} instances!")


Solving: C101.txt
Dimension: 101 nodes
Capacity: 200
Time horizon: [0, 1236]

Creating initial solution...
Initial cost: 4527.78
Initial routes: 58
Time feasible: True

Applying VND...
After VND cost: 1025.28

Applying Simulated Annealing with Tabu Search...

Final cost: 1025.28
Final routes: 58
Time elapsed: 362.12 seconds
Time feasible: True

Optimal cost: 827.30
Gap: 23.93%
✗ Solution exceeds target gap of 7.0%
Solution saved to: solutions_TW/C101_computed.sol

Solving: R101.txt
Dimension: 101 nodes
Capacity: 200
Time horizon: [0, 230]

Creating initial solution...
Initial cost: 2855.88
Initial routes: 43
Time feasible: True

Applying VND...
After VND cost: 1834.34

Applying Simulated Annealing with Tabu Search...

Final cost: 1834.34
Final routes: 43
Time elapsed: 326.13 seconds
Time feasible: True

Optimal cost: 1637.70
Gap: 12.01%
✗ Solution exceeds target gap of 7.0%
Solution saved to: solutions_TW/R101_computed.sol

Solving: RC101.txt
Dimension: 101 nodes
Capacity: 200
Time ho

## Summary Statistics

In [23]:
# Create summary report
import pandas as pd

summary_data = []
for result in results:
    summary_data.append({
        'Instance': result['instance_name'],
        'Computed Cost': f"{result['cost']:.2f}",
        'Optimal Cost': f"{result['optimal_cost']:.2f}" if result['optimal_cost'] else 'N/A',
        'Gap (%)': f"{result['gap_percentage']:.2f}" if result['gap_percentage'] is not None else 'N/A',
        'Routes': result['n_routes'],
        'Time (s)': f"{result['time_seconds']:.2f}"
    })

df_summary = pd.DataFrame(summary_data)
print("\nSummary of VRPTW Results:")
print(df_summary.to_string(index=False))

# Calculate statistics for instances with known optimal
gaps = [r['gap_percentage'] for r in results if r['gap_percentage'] is not None]
if gaps:
    print(f"\n\nGap Statistics:")
    print(f"  Average gap: {np.mean(gaps):.2f}%")
    print(f"  Median gap: {np.median(gaps):.2f}%")
    print(f"  Min gap: {np.min(gaps):.2f}%")
    print(f"  Max gap: {np.max(gaps):.2f}%")
    print(f"  Instances within {config['quality']['target_gap_percentage']}%: {sum(g <= config['quality']['target_gap_percentage'] for g in gaps)}/{len(gaps)}")


Summary of VRPTW Results:
 Instance Computed Cost Optimal Cost Gap (%)  Routes Time (s)
 C101.txt       1025.28       827.30   23.93      58   362.12
 R101.txt       1834.34      1637.70   12.01      43   326.13
RC101.txt       1963.99      1619.80   21.25      34   579.58


Gap Statistics:
  Average gap: 19.06%
  Median gap: 21.25%
  Min gap: 12.01%
  Max gap: 23.93%
  Instances within 7.0%: 0/3


In [24]:
# Save summary to CSV
summary_path = 'solutions_TW/summary_results.csv'
df_summary.to_csv(summary_path, index=False)
print(f"Summary saved to: {summary_path}")

Summary saved to: solutions_TW/summary_results.csv
